In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Continuous bag of words (CBOW) model for text classification
This notebook shows how to use a continuous bag of words (CBOW) model with Pytorch. The task is a text classification problem described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
unpack_dataset()

--2020-02-19 11:49:47--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz’

rotten_imdb.tar.gz  100%[===================>] 507.42K   558KB/s    in 0.9s    

2020-02-19 11:49:49 (558 KB/s) - ‘rotten_imdb.tar.gz’ saved [519599/519599]

x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [72]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [73]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [74]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [75]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [76]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [77]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [78]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [79]:
#!pip install -U spacy

In [80]:
import spacy

In [81]:
# first time run this
#!python3 -m spacy download en

In [82]:
tok = spacy.load('en')

In [83]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [84]:
len(obj_lines)

5000

In [85]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [86]:
test = tok(obj_lines[0])

In [87]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and validation

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [90]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [91]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [93]:
from collections import defaultdict

In [94]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [95]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [96]:
#word_count

In [97]:
len(word_count.keys())

21415

In [98]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [99]:
len(word_count.keys())

4065

In [100]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [101]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [102]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [103]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [104]:
X_train[0]

'will god let her fall or give her a new path ?'

In [105]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

7

In [106]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7])

In [107]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [108]:
encode_sentence(X_train[0])

array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [109]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [110]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [111]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [112]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.6142,  0.2136, -0.6799,  0.6064],
        [ 1.3913,  0.3764, -0.9674, -0.9030],
        [-0.1504, -0.0164,  2.1774,  0.8860],
        [ 1.4216, -0.4580,  1.3691,  0.8674],
        [-0.7108,  1.2483,  0.5496, -1.5263],
        [-1.2288, -0.6853,  0.2598, -0.9845],
        [ 0.4001,  0.3452,  0.5711,  0.5329],
        [ 0.5904, -0.7116, -0.1716, -0.2356],
        [ 1.4376,  1.5275, -0.4301,  0.3887]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [113]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[-0.6142,  0.2136, -0.6799,  0.6064],
         [ 1.4216, -0.4580,  1.3691,  0.8674],
         [-0.6142,  0.2136, -0.6799,  0.6064],
         [-0.7108,  1.2483,  0.5496, -1.5263],
         [-0.6142,  0.2136, -0.6799,  0.6064],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [114]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [115]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [116]:
embed(a)

tensor([[[-0.6142,  0.2136, -0.6799,  0.6064],
         [ 1.4216, -0.4580,  1.3691,  0.8674],
         [-0.6142,  0.2136, -0.6799,  0.6064]],

        [[-0.6142,  0.2136, -0.6799,  0.6064],
         [-0.1504, -0.0164,  2.1774,  0.8860],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [117]:
embed(a).sum(dim=1)

tensor([[ 0.1932, -0.0307,  0.0092,  2.0803],
        [-0.7646,  0.1972,  1.4974,  1.4924]], grad_fn=<SumBackward1>)

In [118]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[ 0.0644, -0.0102,  0.0031,  0.6934],
        [-0.3823,  0.0986,  0.7487,  0.7462]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [119]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [120]:
model = CBOW(vocab_size=5, emb_size=3)

In [121]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.4837,  0.0118, -1.5768],
        [ 0.5992,  0.5553,  0.8514],
        [ 0.8974, -0.0957,  0.5334],
        [-1.9593, -0.5764, -0.4522]], requires_grad=True)

In [122]:
s = s.view(s.shape[0], 1)
model(a, s)

tensor([[-0.9028],
        [-0.7842]], grad_fn=<AddmmBackward>)

# Training the CBOW model 

In [123]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [124]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [125]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.7011229991912842, 0.5065000057220459)

In [126]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (loss.item(), val_loss, val_accuracy))

In [127]:
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.705 val_loss 0.712 val_accuracy 0.500
train_loss 0.701 val_loss 0.619 val_accuracy 0.728
train_loss 0.613 val_loss 0.552 val_accuracy 0.795
train_loss 0.542 val_loss 0.467 val_accuracy 0.854
train_loss 0.448 val_loss 0.381 val_accuracy 0.882
train_loss 0.351 val_loss 0.326 val_accuracy 0.872
train_loss 0.283 val_loss 0.277 val_accuracy 0.892
train_loss 0.225 val_loss 0.251 val_accuracy 0.904
train_loss 0.189 val_loss 0.247 val_accuracy 0.903
train_loss 0.167 val_loss 0.243 val_accuracy 0.910


In [128]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.144 val_loss 0.241 val_accuracy 0.913
train_loss 0.139 val_loss 0.240 val_accuracy 0.913
train_loss 0.136 val_loss 0.240 val_accuracy 0.910
train_loss 0.133 val_loss 0.240 val_accuracy 0.909
train_loss 0.130 val_loss 0.239 val_accuracy 0.910
train_loss 0.126 val_loss 0.238 val_accuracy 0.910
train_loss 0.123 val_loss 0.238 val_accuracy 0.910
train_loss 0.119 val_loss 0.238 val_accuracy 0.911
train_loss 0.116 val_loss 0.238 val_accuracy 0.910
train_loss 0.113 val_loss 0.238 val_accuracy 0.909


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [129]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [130]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [131]:
encode_sentence2(X_train[0])

(array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32), 12)

In [132]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [133]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [134]:
x, y, s

(tensor([[1098,  171,   10,  594,  118,   28,   13,   39, 1386,  417,   51,  130,
          1891,    1,   51,  424, 2638,  959,   51,    1, 1895,   28,   29,  929,
            90, 3126,  696,   10,    1,   51,    1,    1,  171,   14,  113,   20,
             0,    0,    0,    0],
         [  14,  441,  196,   42,   90,    1,   74,  188,  187, 1817,   14,  657,
            20,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  14,  505,   51,  617, 3557,   96,  433, 1077,  796, 1740,  264,   13,
           264,   28,  212, 1432,   14, 2930,   20,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  14, 1198,   51,  438,    1,    1, 2929, 2101,   81,  240,    1, 2930,
            74,  181, 1939,  414,   42,   90, 2262,   20,    0,    0,    0,

In [135]:
model = CBOW(vocab_size=V, emb_size=50)

In [136]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [137]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (train_loss, val_loss, val_accuracy))

In [138]:
train_epocs(model, epochs=10)

train_loss 0.644 val_loss 0.579 val_accuracy 0.775
train_loss 0.493 val_loss 0.416 val_accuracy 0.850
train_loss 0.332 val_loss 0.307 val_accuracy 0.887
train_loss 0.235 val_loss 0.258 val_accuracy 0.898
train_loss 0.182 val_loss 0.238 val_accuracy 0.906
train_loss 0.148 val_loss 0.231 val_accuracy 0.910
train_loss 0.125 val_loss 0.230 val_accuracy 0.910
train_loss 0.107 val_loss 0.233 val_accuracy 0.910
train_loss 0.092 val_loss 0.240 val_accuracy 0.908
train_loss 0.080 val_loss 0.246 val_accuracy 0.909
